# 上下文压缩技术
过滤并压缩检索到的文本块，减少噪声从而提高响应质量，主要作用其实是在检索到的块内删掉无关的句子和段落，也就是chunk内过滤，在上下文窗口中最大化有用信号。


主要使用三种方法：
1. 过滤：分析文档块并仅提取与用户查询直接相关的句子或段落，移除所有无关内容。
2. 摘要：创建文档块的简洁摘要，且仅聚焦与用户查询相关的信息。
3. 抽取：从文档块中精确提取与用户查询相关的完整句子。

In [2]:
import fitz
import os
import re
import json
import numpy as np
from tqdm import tqdm
from zhipuai import ZhipuAI
from dotenv import load_dotenv

load_dotenv()
issues_dir = ".issues"
json_name = "server"
json_path = os.path.join(issues_dir,json_name)
json_path

'.issues\\server'

In [4]:
pdf_path = "../AI_Information.en.zh-CN.pdf"

API_KEY = os.getenv("API_KEY")
client = ZhipuAI(api_key  = API_KEY)

llm_model = os.getenv("llm_model")
embedding_model = os.getenv("embedding_model")

In [5]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [6]:
def extract_text_from_pdf_2(pdf_path):
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化一个空字符串以存储提取的文本

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")  # 从页面中提取文本
        all_text += text  # 将提取的文本追加到 all_text 字符串中

    return all_text  # 返回提取的文本

In [ ]:
read_pdf = extract_text_from_pdf(pdf_path)
#print(read_pdf[:5000])  # 打印前5000个字符以检查内容

In [7]:
def chunk_text(text, n, overlap):
    chunks = []

    chunks = []  
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]
        if chunk:
            chunks.append(chunk)

    return chunks

In [ ]:
demo = "出一款刚配置的台式机，很新，在南京线下实体店刚配的，后面去外地实习用不到了。4060显卡 12400kf.  内存16✖️2   1t固态，多个风扇，黑色海景房，打游戏仿真都绰绰有余。预计价格在4200，非诚勿扰，诚心要的可加v:daji705 。可具体看配置信息"
chunks = chunk_text(demo, 10, 6)
print("分割后的文本块:")
for i, chunk in enumerate(chunks):
    print(f"块 {i + 1}: {chunk}")

In [8]:
class SimpleVectorStore:
    """
    使用 NumPy 实现的轻量级向量存储，包括原始文本、嵌入向量、元数据。
    """
    def __init__(self):
        self.vectors = [] # 存储嵌入向量
        self.texts = [] # 存储对应的原始文本块
        self.metadata = [] # 存储对应的元数据

    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加一个项目。

        Args:
        text (str): 原始文本。
        embedding (List[float]): 嵌入向量。
        metadata (dict, 可选): 额外的元数据。
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        # self.metadata.append(metadata if metadata is not None else {})
        self.metadata.append(metadata or {})

    def similarity_search(self, query_embedding, top_k=5):
        """
        查找与查询嵌入最相似的项目。

        Args:
        query_embedding (List[float]): 查询嵌入向量。
        k (int): 返回的结果数量。

        Returns:
        List[Dict]: 包含文本和元数据的前k个最相似项。
        """
        # 存储向量为空
        if not self.vectors:
            return []
        
        query_vector = np.array(query_embedding)

        #计算余弦相似度
        similarities = []
        for i,vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity)) # 加入索引、相似度元组

        # 按相似度降序排序
        similarities.sort(key=lambda x: x[1], reverse=True)

        results = []
        for i in range(min(top_k, len(similarities))):
            index, similarity = similarities[i]
            results.append({
                "text": self.texts[index],
                "similarity": similarity,
                "metadata": self.metadata[index]
            })

        return results

In [10]:
def create_embeddings(text):
    """
    使用Embedding模型为给定文本创建嵌入向量。

    Args:
        text (str): 要创建嵌入向量的输入文本。

    Returns:
        List[float]: 嵌入向量。
    """
    # 将输入转换为列表来处理字符串
    input_text = text if isinstance(text, list) else [text]

    response = client.embeddings.create(
        model=embedding_model,  
        input=input_text
    )

    if isinstance(text,str):
        return response.data[0].embedding
    
    return [item.embedding for item in response.data]

In [19]:
def process_document(pdf_path,chunk_size=1000, chunk_overlap=200):
    """
    为RAG处理文档。

    Args:
        pdf_path (str): PDF文件的路径。
        chunk_size (int): 每个文本块的大小（以字符为单位）。
        chunk_overlap (int): 文本块之间的重叠大小（以字符为单位）。

    Returns:
        SimpleVectorStore: 包含文档文本块及其嵌入向量的向量存储。
    """
    # 提取、分割、嵌入
    print("提取pdf文本...")
    text = extract_text_from_pdf(pdf_path)

    print("分割文本块...")
    chunks = chunk_text(text, chunk_size, chunk_overlap)
    print(f"总共分割为 {len(chunks)} 个文本块。")

    print("创建嵌入向量...")
    chunk_embeddings = create_embeddings(chunks)
    
    # 创建向量存储，存入文本块、嵌入向量和元数据
    vector_store = SimpleVectorStore()

    for i,(chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        metadata = {
            "index": i,
            "source": pdf_path
        }
        vector_store.add_item(chunk, embedding, metadata)
    
    print(f"向向量存储中添加了 {len(chunks)} 个文本块")
    return vector_store
    

至此，process_document已经完成了基础的pdf的文档处理，包括提取、分块、嵌入向量、向量存储
下面实现上下文压缩的核心部分：过滤、压缩检索内容

# 上下文压缩by llm

In [17]:
def compress_chunk(chunk,query,compression_type="selective"):
    """
    压缩检索到的文本块，仅保留与查询相关的内容。query是压缩检索的依据

    Args:
        chunk (str): 要压缩的文本块
        query (str): 用户查询
        compression_type (str): 压缩类型 ("selective", "summary" 或 "extraction")

    Returns:
        str: 压缩后的文本块
    """

    # 为不同的压缩方法定义系统提示
    if compression_type == "selective":
        system_prompt = """您是专业信息过滤专家。
        您的任务是分析文档块并仅提取与用户查询直接相关的句子或段落，移除所有无关内容。

        输出要求：
        1. 仅保留有助于回答查询的文本
        2. 保持相关句子的原始措辞（禁止改写）
        3. 维持文本的原始顺序
        4. 包含所有相关文本（即使存在重复）
        5. 排除任何与查询无关的文本

        请以纯文本格式输出，不添加任何注释。"""

    elif compression_type == "summary":
        system_prompt = """您是专业摘要生成专家。
        您的任务是创建文档块的简洁摘要，且仅聚焦与用户查询相关的信息。

        输出要求：
        1. 保持简明扼要但涵盖所有相关要素
        2. 仅聚焦与查询直接相关的信息
        3. 省略无关细节
        4. 使用中立、客观的陈述语气

        请以纯文本格式输出，不添加任何注释。"""

    else:  # extraction
        system_prompt = """您是精准信息提取专家。
        您的任务是从文档块中精确提取与用户查询相关的完整句子。

        输出要求：
        1. 仅包含原始文本中的直接引用
        2. 严格保持原始文本的措辞（禁止修改）
        3. 仅选择与查询直接相关的完整句子
        4. 不同句子使用换行符分隔
        5. 不添加任何解释性文字

        请以纯文本格式输出，不添加任何注释。"""

    user_prompt = f"""
        查询: {query}

        文档块:
        {chunk}

        请严格提取与本查询相关的文档块的核心内容。
    """

    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role":"system","content": system_prompt},
            {"role":"user","content": user_prompt}
        ],
        temperature=0.1
    )

    # 从响应中提取压缩后的文本块（str）
    compressed_chunk = response.choices[0].message.content.strip()

    # 计算压缩比率
    original_length = len(chunk)
    compressed_length = len(compressed_chunk)
    compression_ratio = (original_length - compressed_length) / original_length * 100

    return compressed_chunk, compression_ratio


# 批量压缩
一次性压缩多个文本块

In [13]:
def batch_compress_chunks(chunks,query,compression_type="selective"):
    """
    逐个压缩多个文本块。

    Args:
        chunks (List[str]): 要压缩的文本块列表
        query (str): 用户查询
        compression_type (str): 压缩类型 ("selective", "summary", 或 "extraction")

    Returns:
        List[Tuple[str, float]]: 包含压缩比率的压缩文本块列表
    """
    print(f"正在压缩 {len(chunks)} 个文本块...")
    results = []
    sum_original_length = 0
    sum_compressed_length = 0

    for i,chunk in enumerate(chunks):
        print(f"正在压缩文本块 {i+1}/{len(chunks)}...")
        compressed_chunk,compression_ratio = compress_chunk(chunk, query, compression_type)
        results.append((compressed_chunk, compression_ratio))

        sum_original_length += len(chunk)
        sum_compressed_length += len(compressed_chunk)

    # 计算总压缩比率
    total_compression_ratio = (sum_original_length - sum_compressed_length) / sum_original_length * 100
    print(f"总体压缩比率: {total_compression_ratio:.2f}%")

    return results # 返回包含压缩文本块和压缩比率的列表


# 回答

In [20]:
def generate_response(query,context):
    """
    根据查询和上下文生成响应。

    Args:
        query (str): 用户查询
        context (str): 从压缩块中提取的上下文文本，也就是compressed_chunk

    Returns:
        str: 生成的响应
    """
    # 定义系统提示以指导AI的行为
    system_prompt = "您是一个乐于助人的AI助手。请仅根据提供的上下文来回答用户的问题。如果在上下文中找不到答案，请直接说'没有足够的信息'。"

    # 通过组合上下文和查询创建用户提示
    user_prompt = f"""
        上下文:
        {context}

        问题: {query}

        请基于上述上下文内容提供一个全面详尽的答案。
    """

    response = client.chat.completions.create(
        model=llm_model,    
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],          
        temperature=0
    )
    return response.choices[0].message.content


# 上下文压缩的完整RAG执行管道


In [15]:
def rag_with_compression(pdf_path,query,k=10,compression_type="selective"):
    """
    完整的RAG管道，包含上下文压缩。

    Args:
        pdf_path (str): PDF文档的路径
        query (str): 用户查询
        k (int): 初始检索的块数量
        compression_type (str): 压缩类型

    Returns:
        dict: 包括查询、压缩块和响应的结果
    """
    print("\n=== RAG WITH CONTEXTUAL COMPRESSION ===")
    print(f"Query: {query}")
    print(f"Compression type: {compression_type}")

    # 1. 处理文档，完成文档的提取、分块、创建嵌入向量
    vector_store = process_document(pdf_path)

    # 2. 创建 查询 嵌入向量
    query_embedding = create_embeddings(query)
    
    # 3. 检索与查询最相关的k个文本块
    print(f"Retrieving top {k} chunks...")
    results = vector_store.similarity_search(query_embedding, top_k=k)
    #    提取检索到的文本块,results是一个list，三元组
    retrieved_chunks = [result["text"] for result in results]

    # 4. 压缩检索到的文本块
    compressed_results = batch_compress_chunks(retrieved_chunks, query, compression_type)
    #batch_compress_chunks返回compressed_results是（chunk，ratio）
    compressed_chunks = [result[0] for result in compressed_results]
    compression_ratios = [result[1] for result in compressed_results]

    # 过滤掉空的块
    filtered_chunks = [(chunk,ratio) for chunk, ratio in zip(compressed_chunks, compression_ratios) if chunk.strip]

    # 如果过滤后都为空，使用原始块
    if not filtered_chunks:
        print("所有块都被压缩为空，使用原始块。")
        filtered_chunks = [(chunk, 0) for chunk in retrieved_chunks]
    else:
        compressed_chunks, compression_ratios = zip(*filtered_chunks)

    # 5. 将压缩后的块合并为一个上下文字符串
    context = "\n\n".join(compressed_chunks)  # 将压缩后的块合并为一个上下文字符串

    response = generate_response(query, context)
    
    # 6. 返回结果字典
    result = {
        "query": query,
        "original_retrieved_chunks": retrieved_chunks,
        "compressed_chunks": compressed_chunks,
        "compressed_ratios": compression_ratios,
        "context_length_reduction": f"{sum(compression_ratios)/len(compression_ratios):.2f}%",
        "response": response
    }

    print("\n=== RAG RESULT ===")
    print(f"Response: {response}")

    return result


In [21]:
query = "如何评价人工智能对人类的影响?"
result = rag_with_compression(pdf_path, query, k=10, compression_type="selective")


=== RAG WITH CONTEXTUAL COMPRESSION ===
Query: 如何评价人工智能对人类的影响?
Compression type: selective
提取pdf文本...
分割文本块...
总共分割为 13 个文本块。
创建嵌入向量...
向向量存储中添加了 13 个文本块
Retrieving top 10 chunks...
正在压缩 10 个文本块...
正在压缩文本块 1/10...
正在压缩文本块 2/10...
正在压缩文本块 3/10...
正在压缩文本块 4/10...
正在压缩文本块 5/10...
正在压缩文本块 6/10...
正在压缩文本块 7/10...
正在压缩文本块 8/10...
正在压缩文本块 9/10...
正在压缩文本块 10/10...
总体压缩比率: 59.30%

=== RAG RESULT ===
Response: 人工智能对人类的影响是深远和多方面的，既有积极的一面，也存在需要关注和解决的挑战。

积极影响方面：
1. 提高效率：人工智能通过自动化日常任务和优化工作流程，在各个行业中提高了生产效率和服务质量。
2. 创新和创造力：人工智能在艺术、音乐、写作等领域的应用为人类提供了全新的创作工具，激发了新的艺术形式和表达方式。
3. 医疗保健：人工智能在医疗诊断、治疗和药物研发方面的应用，提高了诊断的准确性，加速了新药的发现，实现了更个性化的医疗服务。
4. 城市规划和环境保护：人工智能在智慧城市建设中的应用，如交通系统优化、能源管理改善和环境监测，有助于提升城市运营效率，减少资源浪费，保护环境。
5. 教育个性化：人工智能可以根据每个学生的学习需求和风格提供个性化教育，提高学习效果。
6. 支持决策：人工智能通过分析大量数据，为人类决策提供洞察，增强了决策的科学性和有效性。

挑战和潜在负面影响：
1. 工作岗位流失：人工智能自动化可能导致某些行业的工作岗位减少，特别是那些重复性和低技能的工作。
2. 伦理和偏见问题：如果人工智能系统设计不当，可能会继承并放大训练数据中的偏见，导致不公平和歧视性的结果。
3. 隐私和安全：人工智能系统通常依赖大量数据，这可能引发隐私泄露的风险，同时，人工智能的安全问题也可能带来安全隐患。
4. 透明度和可解释性：一些人工智能系统缺乏透明度，难以解释

# 与标准RAG性能对比

In [ ]:
def standard_rag(pdf_path,query,k=10):
    """
    标准RAG，不包含压缩。

    Args:
        pdf_path (str): PDF文档的路径
        query (str): 用户查询
        k (int): 检索的块数量

    Returns:
        dict: 包括查询、块和响应的结果
    """